In [ ]:
# POde pular essa etapa se os pacotes já estão instalados
!pip install numpy
!pip install scikit-learn
!pip install scikit-image
!pip install librosa
!pip install tqdm
!pip install matplotlib
!pip install tensorflow

In [1]:
#ref: https://www.kaggle.com/code/msripooja/steps-to-convert-audio-clip-to-spectrogram
import os
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.losses import CategoricalCrossentropy
from keras.models import Sequential
from keras.layers import GRU, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Activation, TimeDistributed

#to play audio
import IPython.display as ipd

# Nesse caso você precisa ter no seu diretório uma pasta com o mesmo nome da variável base_dir
base_dir = "audioset"
# E dentro da pasta audioset, você precisa de 3 pastas: carros, motocicletas e motosserras contendo os áudios
audio_carro_path = base_dir + "/carros/"
audio_moto_path = base_dir + "/motocicletas/"  # caminho onde estão os arquivos de audio
audio_serra_path = base_dir + "/motosserras/"

spec_base_path = "./audioset/espectrogramas_com_log_8000Hz"
spec_carro_path = os.path.join(spec_base_path, "carros")
spec_moto_path = os.path.join(spec_base_path, "motocicletas")
spec_serra_path = os.path.join(spec_base_path, "motosserras")
 
os.makedirs(spec_base_path, exist_ok=True)
os.makedirs(spec_carro_path, exist_ok=True)
os.makedirs(spec_moto_path, exist_ok=True)
os.makedirs(spec_serra_path, exist_ok=True)

2024-02-22 13:41:54.630185: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 13:41:55.212214: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 13:41:55.212323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 13:41:55.313885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 13:41:55.542045: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 13:41:55.548017: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
audio_carro_clips = sorted(os.listdir(audio_carro_path))[:250]
print(f"Número de arquivos .wav na pasta {audio_carro_path}= ",len(audio_carro_clips))

audio_moto_clips = sorted(os.listdir(audio_moto_path))[:250]
print(f"Número de arquivos .wav na pasta {audio_moto_path}= ",len(audio_moto_clips))

audio_serra_clips = sorted(os.listdir(audio_serra_path))[:500]
print(f"Número de arquivos .wav na pasta {audio_serra_path}= ",len(audio_serra_clips))

Número de arquivos .wav na pasta ./audioset/carros/=  250
Número de arquivos .wav na pasta ./audioset/motocicletas/=  250
Número de arquivos .wav na pasta ./audioset/motosserras/=  500


In [4]:
#aqui está selecionando o primeiro arquivo da lista, pode ser feito um for para pegar todos de uma vez
#verificar esse sample rate
x_carro, sr_carro = librosa.load(audio_carro_path+audio_carro_clips[0]) 
x_moto, sr_moto = librosa.load(audio_moto_path+audio_moto_clips[4]) 
x_serra, sr_serra = librosa.load(audio_serra_path+audio_serra_clips[6]) 

print(type(x_carro), type(sr_carro))
print(x_carro.shape, sr_carro)

print(type(x_moto), type(sr_moto))
print(x_moto.shape, sr_moto)

print(type(x_serra), type(sr_serra))
print(x_serra.shape, sr_serra)

<class 'numpy.ndarray'> <class 'int'>
(220500,) 22050
<class 'numpy.ndarray'> <class 'int'>
(220500,) 22050
<class 'numpy.ndarray'> <class 'int'>
(220500,) 22050


In [5]:
#converter o audio para espectrograma:
# é nesse amplitude_to_dB que ele está convertendo para pressão sonora. Podemos explorar outras ponderações aqui. Se não tiver implementado, podemos implementar.
def salvar_espectrograma(y, sr, spec_path, log=False):

    X = librosa.stft(y)
    Xdb = librosa.amplitude_to_db(abs(X))

    plt.figure(figsize=(1, 1))

    if log:
        librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log', fmax=8000)
    else:
        librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz', fmax=8000)
    plt.margins(0,0)
    
    plt.axis("off")
    plt.savefig(spec_path, dpi=300, bbox_inches = 'tight',
                pad_inches = 0, transparent=False, facecolor='white')
    plt.close()
    print(f"{spec_path} salvo com sucesso.")

# Salvar imagens de espectrograma
 - Pule esse passo se as imagens já estiverem na pasta

In [5]:
# Salvar espectrogramas de carros
for audio_name in audio_carro_clips:
    x_carro, sr_carro = librosa.load(audio_carro_path + audio_name) 
    audio_name = audio_name.replace(".mp3", ".png")
    spec_path = os.path.join(spec_carro_path, audio_name)
    try:
        salvar_espectrograma(x_carro, sr_carro, spec_path, log=True)
    except:
        print("Erro ao salvar arquivo %s" % spec_path)

# Salvar espectrogramas de motos
for audio_name in audio_moto_clips:
    x_moto, sr_moto = librosa.load(audio_moto_path + audio_name) 
    audio_name = audio_name.replace(".mp3", ".png")
    spec_path = os.path.join(spec_moto_path, audio_name)
    try:
        salvar_espectrograma(x_moto, sr_moto, spec_path, log=True)
    except:
        print("Erro ao salvar arquivo %s" % spec_path)

# Salvar espectrogramas de motosserras
for audio_name in audio_serra_clips:
    x_serra, sr_serra = librosa.load(audio_serra_path + audio_name) 
    audio_name = audio_name.replace(".mp3", ".png")
    spec_path = os.path.join(spec_serra_path, audio_name)
    try:
        salvar_espectrograma(x_serra, sr_serra, spec_path, log=True)
    except:
        print("Erro ao salvar arquivo %s" % spec_path)

./audioset/espectrogramas_com_log_8000Hz/carros/#5) Project ForkenSwift electric car： shifting gears-Liqv3KgXThk.png salvo com sucesso.
Erro ao salvar arquivo ./audioset/espectrogramas_com_log_8000Hz/carros/$150 car jump-1aCyNnYNWvA.png
./audioset/espectrogramas_com_log_8000Hz/carros/'87 Honda Mugen Motul Civic Si Race Car at Costa Di Almafi GT4-28SbkfcA6kE.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/'０７ STREET CAR SHOW  ＃６ 大黒２--NBRp5qsrYQ.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/(16) Car Cam： 2 mobile speed cameras, speed racer & scenic esplanade route-2JAjo7yVCgs.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/(HD) - Forza 4 - Top Gear - Car Test - 1967 Chevrolet Corvette Stingray 427--v2FkuCVLjo.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/(HD) - Forza 4 - Top Gear - Car Test - 1980 Lotus Esprit Turbo-1Z3ciuA3Y3o.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/ca

./audioset/espectrogramas_com_log_8000Hz/carros/(HD) Celebration Exotic Car Rally 4-15-2012-1cFoiMAX0jk.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/(HD) GTA V Epic Highway 5 Star Car Chase (Adder)-4VJSDVXGuqM.png salvo com sucesso.
./audioset/espectrogramas_com_log_8000Hz/carros/(Heat 1) Ahmed Tarek - Red Bull Car Park Drift (CPD) 2013 أحمد طارق دريفت ريد بول-1nayT4f9V-k.png salvo com sucesso.


# Treinamento e teste de rede neural

In [8]:
# Carregar espectrogramas com uma CNN e classificar entre "motosserras e não motosserras"

espectrogramas_carro = sorted(os.listdir(spec_carro_path))
espectrogramas_moto = sorted(os.listdir(spec_moto_path))
espectrogramas_serra = sorted(os.listdir(spec_serra_path))

lista_imagens = []
labels = []

def adicionar_rotulos_imagens(espectrogramas, espectro_path, label):
    for spec in espectrogramas:
        image_path = os.path.join(espectro_path, spec)     
        img = io.imread(image_path)
        lista_imagens.append(img[:, :, :-1])
        # 0 corresponde aos carros e motos
        labels.append(label)

adicionar_rotulos_imagens(espectrogramas_carro, spec_carro_path, 0)
adicionar_rotulos_imagens(espectrogramas_moto, spec_moto_path, 0)
adicionar_rotulos_imagens(espectrogramas_serra, spec_serra_path, 1)

X_train, X_test, y_train, y_test = train_test_split(
    np.array(lista_imagens), np.array(labels), test_size=0.2, random_state=100
)

In [9]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics='accuracy')
model.fit(X_train, y_train, batch_size=32 , epochs=15)

y_pred = model.predict(X_test)[:, 0]
y_pred = np.round(y_pred)
acc = len(np.where(y_pred == y_test)[0]) / len(y_test)
print("Acurácia:", acc)

Epoch 1/15
25/25 [==============================] - 42s 2s/step - loss: 25.1029 - accuracy: 0.5328
Epoch 2/15
25/25 [==============================] - 27s 1s/step - loss: 1.0642 - accuracy: 0.6136
Epoch 3/15
25/25 [==============================] - 26s 1s/step - loss: 0.7240 - accuracy: 0.6237
Epoch 4/15
25/25 [==============================] - 33s 1s/step - loss: 2.7397 - accuracy: 0.6730
Epoch 5/15
25/25 [==============================] - 28s 1s/step - loss: 0.7209 - accuracy: 0.6944
Epoch 6/15
25/25 [==============================] - 27s 1s/step - loss: 0.7915 - accuracy: 0.6439
Epoch 7/15
25/25 [==============================] - 37s 1s/step - loss: 0.6066 - accuracy: 0.7058
Epoch 8/15
25/25 [==============================] - 31s 1s/step - loss: 0.6002 - accuracy: 0.6970
Epoch 9/15
25/25 [==============================] - 32s 1s/step - loss: 0.8523 - accuracy: 0.7311
Epoch 10/15
25/25 [==============================] - 32s 1s/step - loss: 0.5124 - accuracy: 0.7551
Epoch 11/15
25/25 